Imports and Data Loading

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

In [67]:
df1 = pd.read_csv('resale-flat-prices-based-on-approval-date-1990-1999.csv')
df2 = pd.read_csv('resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df3 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df4 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
df5 = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

df = [df1, df2, df3, df4, df5]
df = pd.concat(df)
df.shape

(826581, 11)

In [68]:
df.dropna(inplace=True)
print(df.shape)
print(df.columns)
df.describe()

(117527, 11)
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'remaining_lease'],
      dtype='object')


,floor_area_sqm,lease_commence_date,resale_price
count,117527.000000,117527.000000,1.175270e+05
mean,97.428947,1993.106758,4.385408e+05
std,24.259687,12.338690,1.480609e+05
min,31.000000,1966.000000,1.400000e+05
25%,77.000000,1984.000000,3.330000e+05
50%,95.000000,1993.000000,4.088880e+05
75%,112.000000,2002.000000,5.080000e+05
max,280.000000,2019.000000,1.258000e+06


In [69]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64


Data Transformation

In [10]:
# Transform datetime in string to datetime Y-m format
df['month'] = pd.to_datetime(df['month'], format='%Y-%m')

def get_floor(row):
    floors = row['storey_range'].split(" TO ")
    start = int(floors[0])
    end = int(floors[1])
    return pd.Series([start, end])

# Transform floor range into two integer columns
df[['floor-range-start', 'floor-range-end']]  = df.apply(get_floor, axis=1)


In [12]:
# Transform flat type room number in string to single integer
# 4 ROOM --> (4)

# 5 ROOM --> (5)

# 3 ROOM --> (3)

# 2 ROOM --> (2)

# 1 ROOM --> (1)

# EXECUTIVE --> (6)

# MULTI-GENERATION --> (7)

flat_types = df['flat_type'].unique()
flat_types.sort()
digit_mapping = {flat_type: idx + 1 for idx, flat_type in enumerate(flat_types)}
df['flat_type_digit'] = df['flat_type'].map(digit_mapping)

In [25]:
# Transform range storey into digits
# '04 TO 06' --> 1
#  '07 TO 09' --> 2
#  '10 TO 12' --> 3
#  '01 TO 03' --> 4
#  '13 TO 15' --> 5
#  '16 TO 18' --> 6
#  '19 TO 21' --> 7
#  '22 TO 24' --> 8
#  '25 TO 27' --> 9
#  '28 TO 30' --> 10
#  '34 TO 36' --> 11
#  '31 TO 33' --> 12
#  '37 TO 39' --> 13
#  '40 TO 42' --> 14
#  '46 TO 48' --> 15
#  '43 TO 45' --> 16
#  '49 TO 51' --> 17

d = df['storey_range'].value_counts().to_dict()
storey_ranges = df['storey_range'].unique()
storey_ranges.sort()
storey_mapping = {storey_range: idx + 1 for idx, storey_range in enumerate(storey_ranges)}
df['storey_range_digit'] = df['storey_range'].map(storey_mapping)

In [79]:
# Transform towns to numerical
d = df['town'].value_counts().to_dict()
towns = df['town'].unique()
towns.sort()
town_mapping = {town: idx + 1 for idx, town in enumerate(towns)}
print(town_mapping)
df['town_digit'] = df['town'].map(town_mapping)

{'ANG MO KIO': 1, 'BEDOK': 2, 'BISHAN': 3, 'BUKIT BATOK': 4, 'BUKIT MERAH': 5, 'BUKIT PANJANG': 6, 'BUKIT TIMAH': 7, 'CENTRAL AREA': 8, 'CHOA CHU KANG': 9, 'CLEMENTI': 10, 'GEYLANG': 11, 'HOUGANG': 12, 'JURONG EAST': 13, 'JURONG WEST': 14, 'KALLANG/WHAMPOA': 15, 'MARINE PARADE': 16, 'PASIR RIS': 17, 'PUNGGOL': 18, 'QUEENSTOWN': 19, 'SEMBAWANG': 20, 'SENGKANG': 21, 'SERANGOON': 22, 'TAMPINES': 23, 'TOA PAYOH': 24, 'WOODLANDS': 25, 'YISHUN': 26}


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117527 entries, 0 to 80373
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   town                 117527 non-null  object 
 1   flat_type            117527 non-null  object 
 2   block                117527 non-null  object 
 3   street_name          117527 non-null  object 
 4   storey_range         117527 non-null  object 
 5   floor_area_sqm       117527 non-null  float64
 6   flat_model           117527 non-null  object 
 7   lease_commence_date  117527 non-null  int64  
 8   resale_price         117527 non-null  float64
 9   remaining_lease      117527 non-null  object 
 10  town_digit           117527 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 10.8+ MB


Preprocessing

In [29]:
import seaborn as sns


In [49]:
d = df['storey_range'].value_counts().to_dict()
d

{'04 TO 06': 27658,
 '07 TO 09': 25164,
 '10 TO 12': 22221,
 '01 TO 03': 21521,
 '13 TO 15': 10695,
 '16 TO 18': 4721,
 '19 TO 21': 1999,
 '22 TO 24': 1468,
 '25 TO 27': 793,
 '28 TO 30': 500,
 '34 TO 36': 215,
 '31 TO 33': 211,
 '37 TO 39': 206,
 '40 TO 42': 109,
 '46 TO 48': 21,
 '43 TO 45': 16,
 '49 TO 51': 9}

Train Test Split

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [71]:
dataf = df
dataf.drop(['month'], axis='columns', inplace=True)
df['profession']=df['profession'].astype('category').cat.codes

df.shape

(117527, 10)

In [72]:
X = df.iloc[:, 0:df.shape[1]-1]
Y = df['resale_price']
print(type(Y))
print(f' X shape: {X.shape}')
print(f' Y shape: {Y.shape}')
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f' X train shape: {x_train.shape}')
print(f' Y train shape: {y_train.shape}')

print(f' X test shape: {x_test.shape}')
print(f' Y test shape: {y_test.shape}')

<class 'pandas.core.series.Series'>
 X shape: (117527, 9)
 Y shape: (117527,)
 X train shape: (94021, 9)
 Y train shape: (94021,)
 X test shape: (23506, 9)
 Y test shape: (23506,)


In [73]:
df

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70
1,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65
2,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64
3,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63
4,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64
...,...,...,...,...,...,...,...,...,...,...
80369,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,Improved,1987,440000.0,66 years 03 months
80370,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,Improved,1987,458000.0,65 years 06 months
80371,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,Improved,1987,490000.0,66 years 04 months
80372,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,Maisonette,1987,558000.0,66 years 03 months


Regression Model

In [74]:
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))



In [75]:
df

,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70
1,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65
2,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64
3,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63
4,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64
...,...,...,...,...,...,...,...,...,...,...
80369,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,Improved,1987,440000.0,66 years 03 months
80370,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,Improved,1987,458000.0,65 years 06 months
80371,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,Improved,1987,490000.0,66 years 04 months
80372,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,Maisonette,1987,558000.0,66 years 03 months


In [76]:
regr.fit(x_train, y_train)

ValueError: could not convert string to float: 'QUEENSTOWN'